In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

L, W = 1.0, 0.5  # Length and width of the domain (in cm)
lambda_ = 5.0e9  # Elastic constant (Pa)
mu = 5.0e9       # Shear modulus (Pa)
h = 1.0          # Thickness (cm)

########################
# Two separate PINNs   #
########################
class PINN_Ux(nn.Module):
    def __init__(self):
        super(PINN_Ux, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 1)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        raw_out = self.net(inputs).squeeze(dim=1)

        # Enforce (x+0.5) scaling to mimic the original code
        enforced_ux = (x.flatten() + 0.5) * raw_out
        return enforced_ux.view(-1, 1)

class PINN_Uy(nn.Module):
    def __init__(self):
        super(PINN_Uy, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 128),
            nn.Softplus(beta=10),
            nn.Linear(128, 1)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        raw_out = self.net(inputs).squeeze(dim=1)

        # Enforce (x+0.5) scaling for uy as well
        enforced_uy = (x.flatten() + 0.5) * raw_out

        # The original code sets u_y=0 if x=+0.5
        corner_mask = (x[:, 0] == 0.5)
        enforced_uy[corner_mask] = 0.0
        return enforced_uy.view(-1, 1)

##########################################
# Helper to get [u_x, u_y] from 2 models #
##########################################
def get_u(model_u_x, model_u_y, x, y):
    u_x_pred = model_u_x(x, y)  # shape (N,1)
    u_y_pred = model_u_y(x, y)  # shape (N,1)
    return torch.cat([u_x_pred, u_y_pred], dim=1)  # shape (N,2)

##############################
# Strain and stress (same)   #
##############################
def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x,
                                grad_outputs=torch.ones_like(u_x),
                                retain_graph=True,
                                create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y,
                                grad_outputs=torch.ones_like(u_y),
                                retain_graph=True,
                                create_graph=True)[0]
    u_x_y = torch.autograd.grad(u_x, y,
                                grad_outputs=torch.ones_like(u_x),
                                retain_graph=True,
                                create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x,
                                grad_outputs=torch.ones_like(u_y),
                                retain_graph=True,
                                create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_ / scale_factor) * trace_E + 2.0 * (mu / scale_factor) * E_xx)
    sigma_yy = h * ((lambda_ / scale_factor) * trace_E + 2.0 * (mu / scale_factor) * E_yy)
    sigma_xy = h * (2.0 * (mu / scale_factor) * E_xy)
    return sigma_xx, sigma_yy, sigma_xy

############################################
# Physics (PDE) loss with separate x & y   #
############################################
def physics_loss(model_u_x, model_u_y, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)

    # Combined displacement from the two sub-networks
    u = get_u(model_u_x, model_u_y, x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    # Strain
    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    # Stress
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    # PDE residuals
    sigma_xx_x = torch.autograd.grad(sigma_xx, x,
                                     grad_outputs=torch.ones_like(sigma_xx),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y,
                                     grad_outputs=torch.ones_like(sigma_xy),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y,
                                     grad_outputs=torch.ones_like(sigma_yy),
                                     retain_graph=True,
                                     create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x,
                                     grad_outputs=torch.ones_like(sigma_xy),
                                     retain_graph=True,
                                     create_graph=True)[0]

    # Separate PDE residual for x and y
    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    # Mean squared for each component
    loss_equilibrium_x = torch.mean(residual_x**2)
    loss_equilibrium_y = torch.mean(residual_y**2)

    # Give the y-residual 1e3 heavier weight
    loss_equilibrium = loss_equilibrium_x + 1e3 * loss_equilibrium_y

    return loss_equilibrium, loss_equilibrium_x, loss_equilibrium_y

##################################
# Boundary conditions (same)     #
##################################
def boundary_condition_loss(model_u_x, model_u_y, L, W):
    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    x_A = -L / 2 * torch.ones_like(y_A, requires_grad=True)
    u_A = get_u(model_u_x, model_u_y, x_A, y_A)
    loss_A = torch.mean(u_A**2)

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    x_D = L / 2 * torch.ones_like(y_D, requires_grad=True)
    u_D = get_u(model_u_x, model_u_y, x_D, y_D)
    loss_D = torch.mean((u_D[:, 1:2])**2 + (u_D[:, 0:1] - 0.025 * L)**2)

    # Boundary C (y=0) => enforce u_y=0 at y=0
    x_bottom = torch.linspace(-L/2, L/2, 200).reshape(-1,1).requires_grad_()
    y_bottom = torch.zeros_like(x_bottom, requires_grad=True)
    u_bottom = get_u(model_u_x, model_u_y, x_bottom, y_bottom)
    loss_C = torch.mean(u_bottom[:,1:2]**2)

    # Boundary B: traction-free (sigma_yy = sigma_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_B = W / 2 * torch.ones_like(x_B, requires_grad=True)
    u_B = get_u(model_u_x, model_u_y, x_B, y_B)
    u_B_x, u_B_y = u_B[:, 0:1], u_B[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, y_B)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_yy_B**2 + sigma_xy_B**2)

    return loss_A + loss_D + (loss_B / 1e18) + loss_C

##########################################
# Training: now logs PDE_x and PDE_y     #
##########################################
def train_pinn(model_u_x, model_u_y, optimizer, n_epochs, n_points, L, W):
    loss_history = []

    for epoch in range(n_epochs):
        # Sample points in the top domain
        x = (torch.rand((n_points,1)) * L) - (L/2)  # [-L/2, +L/2]
        y = torch.rand((n_points,1)) * (W/2)       # [0, W/2]

        loss_pde, loss_pde_x, loss_pde_y = physics_loss(model_u_x, model_u_y, x, y)
        loss_bc  = boundary_condition_loss(model_u_x, model_u_y, L, W)

        # Combine PDE + BC
        loss = loss_pde + loss_bc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        if epoch % 500 == 0:
            print(
                f"Epoch {epoch}, "
                f"PDE_x: {loss_pde_x.item():.6e}, "
                f"PDE_y: {loss_pde_y.item():.6e}, "
                f"PDE(total): {loss_pde.item():.6e}, "
                f"BC: {loss_bc.item():.6e}, "
                f"Total: {loss.item():.6e}"
            )

    return loss_history

#####################
# Instantiate model #
#####################
model_u_x = PINN_Ux()
model_u_y = PINN_Uy()

# Single optimizer for both
params = list(model_u_x.parameters()) + list(model_u_y.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)

# Train
n_epochs = 10000
n_points = 1000
loss_history = train_pinn(model_u_x, model_u_y, optimizer, n_epochs, n_points, L, W)

#################
# Compare to CSV #
#################
import pandas as pd

file_path = '/Users/murat/Downloads/data.csv'
comparison_data = pd.read_csv(file_path)

x_vals = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1,1)
y_vals = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1,1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values

y_abs = y_vals.abs()

model_u_x.eval()
model_u_y.eval()
with torch.no_grad():
    u_pred_top = get_u(model_u_x, model_u_y, x_vals, y_abs)

# Mirror for the bottom part
u_pred = u_pred_top.clone()
mask_bottom = (y_vals > 0).view(-1)
u_pred[mask_bottom, 1] *= -1.0

u_x_pred = u_pred[:, 0].numpy()
u_y_pred = u_pred[:, 1].numpy()

comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred
comparison_data['error_u_x'] = abs(u_x_actual - u_x_pred)
comparison_data['error_u_y'] = abs(u_y_actual - u_y_pred)

def safe_percent_error(a, p):
    if np.isclose(a, 0.0):
        return 0.0
    else:
        return abs(a - p) / abs(a) * 100.0

comparison_data['percent_error_u_x'] = [
    safe_percent_error(a, p) for a, p in zip(u_x_actual, u_x_pred)
]
comparison_data['percent_error_u_y'] = [
    safe_percent_error(a, p) for a, p in zip(u_y_actual, u_y_pred)
]

print(comparison_data)

avg_percent_error_x = comparison_data['percent_error_u_x'].mean()
avg_percent_error_y = comparison_data['percent_error_u_y'].mean()

print(f"Average Percent Error for u_x: {avg_percent_error_x:.2f}%")
print(f"Average Percent Error for u_y: {avg_percent_error_y:.2f}%")


Epoch 0, PDE_x: 1.036318e-01, PDE_y: 1.102702e-02, PDE(total): 1.113065e+01, BC: 2.969987e-03, Total: 1.113362e+01
Epoch 500, PDE_x: 2.550296e-04, PDE_y: 2.381111e-06, PDE(total): 2.636141e-03, BC: 3.288079e-07, Total: 2.636470e-03
Epoch 1000, PDE_x: 1.029840e-04, PDE_y: 1.492643e-06, PDE(total): 1.595627e-03, BC: 3.018668e-07, Total: 1.595929e-03
Epoch 1500, PDE_x: 9.108472e-05, PDE_y: 7.151236e-07, PDE(total): 8.062084e-04, BC: 2.789385e-07, Total: 8.064873e-04
Epoch 2000, PDE_x: 7.870488e-05, PDE_y: 4.067912e-07, PDE(total): 4.854960e-04, BC: 2.665641e-07, Total: 4.857626e-04
Epoch 2500, PDE_x: 6.335396e-05, PDE_y: 2.677481e-07, PDE(total): 3.311021e-04, BC: 2.586939e-07, Total: 3.313608e-04
Epoch 3000, PDE_x: 5.953158e-05, PDE_y: 1.569299e-07, PDE(total): 2.164615e-04, BC: 2.518604e-07, Total: 2.167133e-04
Epoch 3500, PDE_x: 4.441489e-05, PDE_y: 1.099061e-07, PDE(total): 1.543210e-04, BC: 2.447799e-07, Total: 1.545658e-04
Epoch 4000, PDE_x: 4.202103e-05, PDE_y: 9.436697e-08, PDE(to